# Análisis de Optimización: Implementaciones Extras

Este notebook contiene implementaciones adicionales y análisis complementarios para el estudio de la función $f(x, y) = \log(x^2 + y^2 + 1) \cdot \arctan(x^2 + y^2)$.

**Contenido:**
1. Métodos Quasi-Newton (BFGS, DFP)
2. Algoritmos de Región de Confianza
3. Métodos de Penalidad para Problemas con Restricciones
4. Análisis de Sensibilidad Avanzado
5. Visualizaciones Extendidas
6. Experimentos Adicionales

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from scipy.optimize import minimize, BFGS, NonlinearConstraint
import time
import warnings
from numba import jit
import pandas as pd
# sklearn es opcional para algunas utilidades; manejar ausencia gracilmente
try:
    from sklearn.preprocessing import StandardScaler
    SKLEARN_AVAILABLE = True
except Exception:
    SKLEARN_AVAILABLE = False
    StandardScaler = None
    print("Warning: scikit-learn not installed. To enable optional utilities install it with: pip install scikit-learn")

warnings.filterwarnings('ignore')

# Configuración para mejores gráficas
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['font.size'] = 12
sns.set_palette("husl")

ModuleNotFoundError: No module named 'sklearn'

## 1. Métodos Quasi-Newton

Implementaciones de BFGS y DFP como funciones reutilizables.

In [ ]:
def bfgs_method(f, grad_f, x0, max_iter=1000, tol=1e-8):
    """Implementación del método BFGS (Broyden-Fletcher-Goldfarb-Shanno)
    - Aproximación de la Hessiana usando actualizaciones de rango 2
    - Convergencia superlineal
    - No requiere cálculo explícito de la Hessiana
    """
    x = np.array(x0, dtype=float)
    n = len(x)
    H = np.eye(n)
    trajectory = [x.copy()]
    for i in range(max_iter):
        grad = np.array(grad_f(x[0], x[1]))
        grad_norm = np.linalg.norm(grad)
        if grad_norm < tol:
            break
        d = -H @ grad
        # Búsqueda lineal (backtracking)
        alpha = 1.0
        rho = 0.5
        c = 1e-4
        for _ in range(20):
            x_new = x + alpha * d
            if f(x_new[0], x_new[1]) <= f(x[0], x[1]) + c * alpha * grad @ d:
                break
            alpha *= rho
        x_old = x.copy()
        x = x + alpha * d
        s = x - x_old
        grad_new = np.array(grad_f(x[0], x[1]))
        y = grad_new - grad
        if np.abs(s @ y) > 1e-12:
            rho_bfgs = 1.0 / (s @ y)
            I = np.eye(n)
            H = (I - rho_bfgs * np.outer(s, y)) @ H @ (I - rho_bfgs * np.outer(y, s)) + rho_bfgs * np.outer(s, s)
        trajectory.append(x.copy())
    return x, trajectory

def dfp_method(f, grad_f, x0, max_iter=1000, tol=1e-8):
    """Implementación del método DFP (Davidon-Fletcher-Powell)
    - Aproximación de la inversa de la Hessiana
    - Convergencia superlineal
    """
    x = np.array(x0, dtype=float)
    n = len(x)
    B_inv = np.eye(n)
    trajectory = [x.copy()]
    for i in range(max_iter):
        grad = np.array(grad_f(x[0], x[1]))
        grad_norm = np.linalg.norm(grad)
        if grad_norm < tol:
            break
        d = -B_inv @ grad
        alpha = 1.0
        rho = 0.5
        c = 1e-4
        for _ in range(20):
            x_new = x + alpha * d
            if f(x_new[0], x_new[1]) <= f(x[0], x[1]) + c * alpha * grad @ d:
                break
            alpha *= rho
        x_old = x.copy()
        x = x + alpha * d
        s = x - x_old
        grad_new = np.array(grad_f(x[0], x[1]))
        y = grad_new - grad
        if np.abs(s @ y) > 1e-12:
            B_inv = B_inv + np.outer(s, s) / (s @ y) - (B_inv @ np.outer(y, y) @ B_inv) / (y @ B_inv @ y)
        trajectory.append(x.copy())
    return x, trajectory

## 2. Métodos de Región de Confianza

Implementación del método Dogleg y utilidades.

In [ ]:
def find_tau_dogleg(p_cauchy, p_newton, delta):
    u = p_cauchy
    v = p_newton - p_cauchy
    a = v @ v
    b = 2 * (u @ v)
    c = u @ u - delta ** 2
    discriminant = b ** 2 - 4 * a * c
    if discriminant < 0:
        return 1.0
    tau1 = (-b + np.sqrt(discriminant)) / (2 * a)
    tau2 = (-b - np.sqrt(discriminant)) / (2 * a)
    valid_taus = [tau for tau in [tau1, tau2] if 0 <= tau <= 1]
    if valid_taus:
        return max(valid_taus)
    else:
        return 1.0

def trust_region_dogleg(f, grad_f, hess_f, x0, max_iter=100, tol=1e-8, delta0=1.0):
    x = np.array(x0, dtype=float)
    delta = delta0
    eta = 0.1
    trajectory = [x.copy()]
    for i in range(max_iter):
        grad = np.array(grad_f(x[0], x[1]))
        H = hess_f(x[0], x[1])
        if np.linalg.norm(grad) < tol:
            break
        eigenvalues = np.linalg.eigvals(H)
        if np.min(eigenvalues) < 1e-8:
            H_reg = H + 1e-8 * np.eye(2)
        else:
            H_reg = H
        try:
            p_newton = -np.linalg.solve(H_reg, grad)
        except np.linalg.LinAlgError:
            p_newton = -np.linalg.pinv(H_reg) @ grad
        p_cauchy = - (grad @ grad) / (grad @ H @ grad) * grad
        if np.linalg.norm(p_newton) <= delta:
            p = p_newton
        elif np.linalg.norm(p_cauchy) >= delta:
            p = (delta / np.linalg.norm(p_cauchy)) * p_cauchy
        else:
            tau = find_tau_dogleg(p_cauchy, p_newton, delta)
            p = p_cauchy + tau * (p_newton - p_cauchy)
        actual_reduction = f(x[0], x[1]) - f(x[0] + p[0], x[1] + p[1])
        predicted_reduction = -grad @ p - 0.5 * p @ H @ p
        if predicted_reduction == 0:
            rho = 0
        else:
            rho = actual_reduction / predicted_reduction
        if rho < 0.25:
            delta *= 0.25
        elif rho > 0.75 and np.linalg.norm(p) == delta:
            delta = min(2 * delta, 10.0)
        if rho > eta:
            x = x + p
        trajectory.append(x.copy())
    return x, trajectory

## 3. Métodos de Penalidad para Problemas con Restricciones

Penalidad cuadrática y Lagrangiano aumentado.

In [ ]:
def quadratic_penalty_method(f, grad_f, constraints, x0, mu0=1.0, max_iter=100, tol=1e-6):
    x = np.array(x0, dtype=float)
    mu = mu0
    trajectory = [x.copy()]
    penalty_parameters = [mu]
    for k in range(max_iter):
        def P(x_val):
            penalty = 0
            for h, _ in constraints:
                penalty += h(x_val[0], x_val[1]) ** 2
            return f(x_val[0], x_val[1]) + mu * penalty
        def grad_P(x_val):
            grad_penalty = np.zeros(2)
            for h, grad_h in constraints:
                h_val = h(x_val[0], x_val[1])
                grad_h_val = np.array(grad_h(x_val[0], x_val[1]))
                grad_penalty += 2 * h_val * grad_h_val
            grad_f_val = np.array(grad_f(x_val[0], x_val[1]))
            return grad_f_val + mu * grad_penalty
        x_opt, _ = bfgs_method(lambda x, y: P([x, y]), 
                              lambda x, y: grad_P([x, y]), 
                              x, max_iter=1000, tol=1e-6)
        max_constraint_violation = 0
        for h, _ in constraints:
            max_constraint_violation = max(max_constraint_violation, 
                                         abs(h(x_opt[0], x_opt[1])))
        if max_constraint_violation < tol:
            break
        mu *= 10
        x = x_opt
        trajectory.append(x.copy())
    return x, trajectory, penalty_parameters

def augmented_lagrangian(f, grad_f, constraints, x0, lambda0=None, mu0=1.0, 
                        max_iter=100, tol=1e-6):
    x = np.array(x0, dtype=float)
    mu = mu0
    if lambda0 is None:
        lambda_vec = np.zeros(len(constraints))
    else:
        lambda_vec = np.array(lambda0)
    trajectory = [x.copy()]
    lambda_history = [lambda_vec.copy()]
    for k in range(max_iter):
        def L_aug(x_val):
            penalty = 0
            for i, (h, _) in enumerate(constraints):
                h_val = h(x_val[0], x_val[1])
                penalty += lambda_vec[i] * h_val + (mu/2) * h_val ** 2
            return f(x_val[0], x_val[1]) + penalty
        def grad_L_aug(x_val):
            grad_penalty = np.zeros(2)
            for i, (h, grad_h) in enumerate(constraints):
                h_val = h(x_val[0], x_val[1])
                grad_h_val = np.array(grad_h(x_val[0], x_val[1]))
                grad_penalty += (lambda_vec[i] + mu * h_val) * grad_h_val
            grad_f_val = np.array(grad_f(x_val[0], x_val[1]))
            return grad_f_val + grad_penalty
        x_opt, _ = bfgs_method(lambda x, y: L_aug([x, y]), 
                              lambda x, y: grad_L_aug([x, y]), 
                              x, max_iter=1000, tol=1e-6)
        for i, (h, _) in enumerate(constraints):
            lambda_vec[i] += mu * h(x_opt[0], x_opt[1])
        max_constraint_violation = 0
        for h, _ in constraints:
            max_constraint_violation = max(max_constraint_violation, 
                                         abs(h(x_opt[0], x_opt[1])))
        if max_constraint_violation < tol:
            break
        if max_constraint_violation > 0.25 * tol:
            mu *= 10
        x = x_opt
        trajectory.append(x.copy())
        lambda_history.append(lambda_vec.copy())
    return x, trajectory, lambda_history

## 4. Análisis de Sensibilidad Avanzado

Funciones auxiliares para analizar la sensibilidad del óptimo ante parámetros.

In [ ]:
def sensitivity_analysis(f, grad_f, hess_f, x_opt, parameters, delta=0.01):
    H = hess_f(x_opt[0], x_opt[1])
    if np.linalg.cond(H) > 1e12:
        print("Advertencia: Hessiana mal condicionada, usando pseudo-inversa")
        H_inv = np.linalg.pinv(H)
    else:
        H_inv = np.linalg.inv(H)
    sensitivities = {}
    for param_name, param_value in parameters.items():
        if param_name == 'learning_rate':
            grad_perturbed = finite_difference_gradient(f, x_opt, delta, param_value)
        elif param_name == 'regularization':
            grad_perturbed = finite_difference_regularization(f, x_opt, delta, param_value)
        else:
            grad_perturbed = np.zeros(2)
        sensitivity = -H_inv @ grad_perturbed
        sensitivities[param_name] = sensitivity
    return sensitivities

def finite_difference_gradient(f, x, delta, learning_rate):
    grad_plus = np.zeros(2)
    for i in range(2):
        x_plus = x.copy()
        x_plus[i] += delta
        x_next_plus = x_plus - (learning_rate + delta) * np.array([1, 1])  # Gradiente simulado
        grad_plus[i] = (f(x_next_plus[0], x_next_plus[1]) - f(x[0], x[1])) / delta
    return grad_plus

def finite_difference_regularization(f, x, delta, regularization):
    grad_plus = np.zeros(2)
    for i in range(2):
        x_plus = x.copy()
        x_plus[i] += delta
        reg_effect_plus = regularization * np.linalg.norm(x_plus) ** 2
        reg_effect = regularization * np.linalg.norm(x) ** 2
        grad_plus[i] = ((f(x_plus[0], x_plus[1]) + reg_effect_plus) - 
                       (f(x[0], x[1]) + reg_effect)) / delta
    return grad_plus

def convergence_rate_analysis(trajectories, algorithm_names):
    convergence_data = {}
    for name, trajectory in zip(algorithm_names, trajectories):
        errors = []
        for point in trajectory:
            error = np.linalg.norm(point - np.array([0, 0]))
            errors.append(error)
        if len(errors) > 1:
            rates = []
            for i in range(1, len(errors)):
                if errors[i-1] > 1e-12:
                    rate = errors[i] / errors[i-1]
                    rates.append(rate)
            avg_rate = np.mean(rates) if rates else 0
            std_rate = np.std(rates) if rates else 0
        else:
            avg_rate = 0
            std_rate = 0
        convergence_data[name] = {
            'final_error': errors[-1] if errors else float('inf'),
            'iterations': len(trajectory),
            'avg_convergence_rate': avg_rate,
            'std_convergence_rate': std_rate,
            'errors': errors
        }
    return convergence_data

## 5. Visualizaciones Extendidas

Funciones para comparar convergencia y visualizar paisajes 3D con trayectorias.

In [ ]:
def plot_convergence_comparison(convergence_data):
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    for name, data in convergence_data.items():
        axes[0, 0].semilogy(data['errors'], label=name, linewidth=2)
    axes[0, 0].set_xlabel('Iteración')
    axes[0, 0].set_ylabel('Error (log scale)')
    axes[0, 0].set_title('Convergencia de Algoritmos')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    names = list(convergence_data.keys())
    avg_rates = [convergence_data[name]['avg_convergence_rate'] for name in names]
    std_rates = [convergence_data[name]['std_convergence_rate'] for name in names]
    bars = axes[0, 1].bar(names, avg_rates, yerr=std_rates, capsize=5, alpha=0.7)
    axes[0, 1].set_ylabel('Tasa de Convergencia Promedio')
    axes[0, 1].set_title('Análisis de Tasas de Convergencia')
    axes[0, 1].tick_params(axis='x', rotation=45)
    for bar, rate in zip(bars, avg_rates):
        height = bar.get_height()
        axes[0, 1].text(bar.get_x() + bar.get_width()/2., height + 0.01, f'{rate:.3f}', ha='center', va='bottom')
    iterations = [convergence_data[name]['iterations'] for name in names]
    final_errors = [convergence_data[name]['final_error'] for name in names]
    scatter = axes[1, 0].scatter(iterations, final_errors, s=100, alpha=0.7)
    for i, name in enumerate(names):
        axes[1, 0].annotate(name, (iterations[i], final_errors[i]), xytext=(5, 5), textcoords='offset points')
    axes[1, 0].set_xlabel('Número de Iteraciones')
    axes[1, 0].set_ylabel('Error Final')
    axes[1, 0].set_title('Eficiencia vs Precisión')
    axes[1, 0].set_yscale('log')
    axes[1, 0].grid(True, alpha=0.3)
    sensitivity_matrix = np.array([ [convergence_data[name]['avg_convergence_rate'], convergence_data[name]['std_convergence_rate']] for name in names])
    im = axes[1, 1].imshow(sensitivity_matrix, cmap='viridis', aspect='auto')
    axes[1, 1].set_xticks([0, 1])
    axes[1, 1].set_xticklabels(['Tasa Promedio', 'Desviación'])
    axes[1, 1].set_yticks(range(len(names)))
    axes[1, 1].set_yticklabels(names)
    axes[1, 1].set_title('Mapa de Sensibilidad')
    for i in range(len(names)):
        for j in range(2):
            axes[1, 1].text(j, i, f'{sensitivity_matrix[i, j]:.3f}', ha="center", va="center", color="w")
    plt.colorbar(im, ax=axes[1, 1])
    plt.tight_layout()
    plt.show()

def plot_3d_landscape_with_trajectories(f, trajectories, algorithm_names, x_range=(-2, 2), y_range=(-2, 2)):
    fig = plt.figure(figsize=(15, 10))
    x = np.linspace(x_range[0], x_range[1], 50)
    y = np.linspace(y_range[0], y_range[1], 50)
    X, Y = np.meshgrid(x, y)
    Z = f(X, Y)
    ax1 = fig.add_subplot(121, projection='3d')
    surf = ax1.plot_surface(X, Y, Z, cmap='viridis', alpha=0.7, linewidth=0, antialiased=True)
    colors = ['red', 'blue', 'green', 'orange', 'purple']
    for i, (trajectory, name) in enumerate(zip(trajectories, algorithm_names)):
        if len(trajectory) > 0:
            x_vals = [point[0] for point in trajectory]
            y_vals = [point[1] for point in trajectory]
            z_vals = [f(point[0], point[1]) for point in trajectory]
            ax1.plot(x_vals, y_vals, z_vals, color=colors[i % len(colors)], linewidth=3, marker='o', markersize=4, label=name)
    ax1.set_xlabel('X')
    ax1.set_ylabel('Y')
    ax1.set_zlabel('f(X, Y)')
    ax1.set_title('Paisaje 3D con Trayectorias de Optimización')
    ax1.legend()
    ax2 = fig.add_subplot(122)
    contour = ax2.contour(X, Y, Z, levels=20, alpha=0.6)
    ax2.clabel(contour, inline=True, fontsize=8)
    for i, (trajectory, name) in enumerate(zip(trajectories, algorithm_names)):
        if len(trajectory) > 0:
            x_vals = [point[0] for point in trajectory]
            y_vals = [point[1] for point in trajectory]
            ax2.plot(x_vals, y_vals, color=colors[i % len(colors)], linewidth=2, marker='o', markersize=4, label=name)
    ax2.set_xlabel('X')
    ax2.set_ylabel('Y')
    ax2.set_title('Mapa de Contorno con Trayectorias')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

## 6. Experimentos Adicionales

Funciones para ejecutar experimentos extendidos y con restricciones.

In [ ]:
def run_extended_experiments(f, grad_f, hess_f):
    extended_test_points = [ (15, -25), (-30, -40), (0.001, 0.001), (75, -75), (-0.1, 0.2), (200, 50) ]
    all_results = {}
    for point in extended_test_points:
        print(f"\n=== Experimentos desde punto: {point} ===")
        point_results = {}
        start_time = time.time()
        x_opt_bfgs, traj_bfgs = bfgs_method(f, grad_f, point)
        bfgs_time = time.time() - start_time
        start_time = time.time()
        x_opt_dfp, traj_dfp = dfp_method(f, grad_f, point)
        dfp_time = time.time() - start_time
        start_time = time.time()
        x_opt_trust, traj_trust = trust_region_dogleg(f, grad_f, hess_f, point)
        trust_time = time.time() - start_time
        point_results['BFGS'] = {'x_opt': x_opt_bfgs, 'trajectory': traj_bfgs, 'time': bfgs_time, 'iterations': len(traj_bfgs), 'final_value': f(x_opt_bfgs[0], x_opt_bfgs[1]) }
        point_results['DFP'] = {'x_opt': x_opt_dfp, 'trajectory': traj_dfp, 'time': dfp_time, 'iterations': len(traj_dfp), 'final_value': f(x_opt_dfp[0], x_opt_dfp[1]) }
        point_results['Trust Region'] = {'x_opt': x_opt_trust, 'trajectory': traj_trust, 'time': trust_time, 'iterations': len(traj_trust), 'final_value': f(x_opt_trust[0], x_opt_trust[1]) }
        all_results[point] = point_results
        for method, results in point_results.items():
            success = "ÉXITO" if results['final_value'] < 1e-6 else "FALLO"
            print(f"  {method:15}: {results['iterations']:3d} iter, {results['time']:.4f}s, f(x) = {results['final_value']:.2e} [{success}]")
    return all_results

def constrained_optimization_experiments(f, grad_f):
    def constraint1(x, y):
        return x + y - 1
    def grad_constraint1(x, y):
        return np.array([1, 1])
    def constraint2(x, y):
        return x**2 + y**2 - 4
    def grad_constraint2(x, y):
        return np.array([2*x, 2*y])
    constraints = [(constraint1, grad_constraint1), (constraint2, grad_constraint2)]
    constrained_points = [(3, 3), (-1, -1), (0, 2), (2, 0)]
    constrained_results = {}
    for point in constrained_points:
        print(f"\n=== Optimización con Restricciones desde: {point} ===")
        x_opt_penalty, traj_penalty, mu_history = quadratic_penalty_method(f, grad_f, constraints, point)
        x_opt_auglag, traj_auglag, lambda_history = augmented_lagrangian(f, grad_f, constraints, point)
        penalty_feasible = all(abs(constraint(x_opt_penalty[0], x_opt_penalty[1])) < 1e-6 for constraint, _ in constraints)
        auglag_feasible = all(abs(constraint(x_opt_auglag[0], x_opt_auglag[1])) < 1e-6 for constraint, _ in constraints)
        constrained_results[point] = {'Penalty': {'x_opt': x_opt_penalty, 'feasible': penalty_feasible, 'time': None, 'iterations': len(traj_penalty), 'final_value': f(x_opt_penalty[0], x_opt_penalty[1])},
                                    'Augmented Lagrangian': {'x_opt': x_opt_auglag, 'feasible': auglag_feasible, 'time': None, 'iterations': len(traj_auglag), 'final_value': f(x_opt_auglag[0], x_opt_auglag[1])} }
        for method, results in constrained_results[point].items():
            feasible_str = "FACTIBLE" if results['feasible'] else "NO FACTIBLE"
            print(f"  {method:20}: {results['iterations']:3d} iter, f(x) = {results['final_value']:.2e} [{feasible_str}]")
    return constrained_results

## 7. Análisis de Rendimiento y Escalabilidad

Benchmarking y análisis de escalabilidad para los algoritmos implementados.

In [ ]:
def performance_benchmark(f, grad_f, hess_f, algorithms, test_points, n_runs=10):
    benchmark_results = {}
    for algo_name, algo_func in algorithms.items():
        algo_results = {'total_time': 0, 'success_rate': 0, 'avg_iterations': 0, 'avg_precision': 0, 'point_results': {}}
        for point in test_points:
            point_times = []
            point_successes = []
            point_iterations = []
            point_precisions = []
            for run in range(n_runs):
                start_time = time.time()
                try:
                    if algo_name in ['BFGS', 'DFP']:
                        x_opt, trajectory = algo_func(f, grad_f, point)
                    elif algo_name == 'Trust Region':
                        x_opt, trajectory = algo_func(f, grad_f, hess_f, point)
                    else:
                        x_opt, trajectory = algo_func(point)
                    run_time = time.time() - start_time
                    final_value = f(x_opt[0], x_opt[1])
                    success = final_value < 1e-6
                    point_times.append(run_time)
                    point_successes.append(success)
                    point_iterations.append(len(trajectory))
                    point_precisions.append(final_value)
                except Exception as e:
                    point_times.append(float('inf'))
                    point_successes.append(False)
                    point_iterations.append(float('inf'))
                    point_precisions.append(float('inf'))
            algo_results['point_results'][point] = {'avg_time': np.mean(point_times), 'std_time': np.std(point_times), 'success_rate': np.mean(point_successes), 'avg_iterations': np.mean(point_iterations), 'avg_precision': np.mean(point_precisions)}
            algo_results['total_time'] += np.sum(point_times)
            algo_results['success_rate'] += np.sum(point_successes)
            algo_results['avg_iterations'] += np.sum(point_iterations)
            algo_results['avg_precision'] += np.sum(point_precisions)
        total_points = len(test_points) * n_runs
        algo_results['success_rate'] /= total_points
        algo_results['avg_iterations'] /= total_points
        algo_results['avg_precision'] /= total_points
        benchmark_results[algo_name] = algo_results
    return benchmark_results

def scalability_analysis(f, grad_f, hess_f, base_points, scale_factors):
    scalability_results = {}
    for scale in scale_factors:
        scaled_points = [(scale * x, scale * y) for x, y in base_points]
        scale_results = {}
        algorithms = {'BFGS': lambda p: bfgs_method(f, grad_f, p), 'DFP': lambda p: dfp_method(f, grad_f, p), 'Trust Region': lambda p: trust_region_dogleg(f, grad_f, hess_f, p)}
        for algo_name, algo_func in algorithms.items():
            success_count = 0
            total_time = 0
            total_iterations = 0
            for point in scaled_points:
                try:
                    start_time = time.time()
                    x_opt, trajectory = algo_func(point)
                    end_time = time.time()
                    if f(x_opt[0], x_opt[1]) < 1e-6:
                        success_count += 1
                    total_time += end_time - start_time
                    total_iterations += len(trajectory)
                except Exception as e:
                    pass
            scale_results[algo_name] = {'success_rate': success_count / len(scaled_points), 'avg_time': total_time / len(scaled_points), 'avg_iterations': total_iterations / len(scaled_points)}
        scalability_results[scale] = scale_results
    return scalability_results

## 8. Funciones de Resumen y Reportes

Funciones para generar reportes y guardar/cargar resultados.

In [ ]:
def generate_comprehensive_report(benchmark_results, scalability_results, constrained_results, convergence_data):
    print("=" * 80)
    print("REPORTE COMPREHENSIVO - ANÁLISIS DE OPTIMIZACIÓN")
    print("=" * 80)
    # Resumen de benchmark
    benchmark_df = []
    for algo_name, results in benchmark_results.items():
        benchmark_df.append({'Algoritmo': algo_name, 'Tasa Éxito': f"{results['success_rate']:.2%}", 'Tiempo Promedio': f"{results['total_time'] / (len(results['point_results']) * 10):.4f}s", 'Iteraciones Promedio': f"{results['avg_iterations']:.1f}", 'Precisión Promedio': f"{results['avg_precision']:.2e}"})
    for row in benchmark_df:
        print(f"  {row['Algoritmo']:15} | {row['Tasa Éxito']:8} | {row['Tiempo Promedio']:12} | {row['Iteraciones Promedio']:8} | {row['Precisión Promedio']:12}")
    # Recomendaciones (resumen simplificado)
    best_success = max(benchmark_results.items(), key=lambda x: x[1]['success_rate'])
    best_time = min(benchmark_results.items(), key=lambda x: x[1]['total_time'])
    best_precision = min(benchmark_results.items(), key=lambda x: x[1]['avg_precision'])
    print(f"  Mayor robustez: {best_success[0]} ({best_success[1]['success_rate']:.2%} éxito)")
    print(f"  Mayor velocidad: {best_time[0]}")
    print(f"  Mayor precisión: {best_precision[0]}")

def save_results_to_file(results, filename="optimization_results_extras.npz"):
    serializable_results = {}
    for key, value in results.items():
        if isinstance(value, dict):
            nested_dict = {}
            for nested_key, nested_value in value.items():
                if hasattr(nested_value, '__iter__') and not isinstance(nested_value, str):
                    nested_dict[nested_key] = np.array(nested_value)
                else:
                    nested_dict[nested_key] = nested_value
            serializable_results[key] = nested_dict
        elif hasattr(value, '__iter__') and not isinstance(value, str):
            serializable_results[key] = np.array(value)
        else:
            serializable_results[key] = value
    np.savez(filename, **serializable_results)
    print(f"Resultados guardados en {filename}")

def load_results_from_file(filename="optimization_results_extras.npz"):
    try:
        data = np.load(filename, allow_pickle=True)
        results = {key: data[key] for key in data.files}
        print(f"Resultados cargados desde {filename}")
        return results
    except FileNotFoundError:
        print(f"Archivo {filename} no encontrado")
        return None

## Ejecución Principal de Experimentos Extendidos

Bloque principal para correr experimentos cuando se ejecuta como script.

In [ ]:
# Ejemplo de uso (descomentar y ajustar según la disponibilidad de `f, grad_f, hess_f`)
# from optimization_analysis import f, grad_f, hess_f
# extended_results = run_extended_experiments(f, grad_f, hess_f)
# constrained_results = constrained_optimization_experiments(f, grad_f)
# test_points = [(10, 10), (-50, 50), (0.5, 0.5), (-10, -20)]
# algorithms = {'BFGS': lambda p: bfgs_method(f, grad_f, p), 'DFP': lambda p: dfp_method(f, grad_f, p), 'Trust Region': lambda p: trust_region_dogleg(f, grad_f, hess_f, p)}
# benchmark_results = performance_benchmark(f, grad_f, hess_f, algorithms, test_points, n_runs=5)
# base_points = [(1, 1), (2, 2), (5, 5), (10, 10)]
# scalability_results = scalability_analysis(f, grad_f, hess_f, base_points, [1,10,100])
# convergence_data = {}
# generate_comprehensive_report(benchmark_results, scalability_results, constrained_results, convergence_data)
# save_results_to_file({'extended_results': extended_results, 'constrained_results': constrained_results})